In [ ]:
!bash <(wget -O - https://raw.githubusercontent.com/Athlon1600/php-proxy-installer/master/install.sh)

In [ ]:
!wget -q "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz" -O ngrok.gz
!gunzip "ngrok.gz"
!tar -xf ngrok
!mv ngrok /bin/ngrok
!ngrok authtoken [enter-authtoken] 2>&1 # Enter your auth token here first

In [ ]:
!php -r "copy('https://getcomposer.org/installer', 'composer-setup.php');" 2>&1
!php -r "if (hash_file('sha384', 'composer-setup.php') === '906a84df04cea2aa72f40b5f787e49f22d4c2f19492ac310e8cba5b96ac8b64115ac402c8cd292b8a03482574915d1a8') { echo 'Installer verified'; } else { echo 'Installer corrupt'; unlink('composer-setup.php'); } echo PHP_EOL;" 2>&1
!php composer-setup.php 2>&1
!php -r "unlink('composer-setup.php');" 2>&1
!mv composer.phar /bin/composer 2>&1

In [ ]:
!composer create-project athlon1600/php-proxy-app:dev-master 2>&1

In [ ]:
with open('php-proxy-app/config.php', 'r') as f:
    data = f.read().split('\n')

newdat = ''

for line in data:
    if line.startswith("$config['app_key'] = '';"):
        newdat += "$config['app_key'] = 'key28490';\n"
    else:
        newdat += line+'\n'

with open('php-proxy-app/config.php', 'w') as f:
    f.write(newdat)

In [ ]:
!cd php-proxy-app ; wget -q "https://findicons.com/icon/download/direct/169016/php/128/ico" -O favicon.ico
!cd php-proxy-app ; chmod +rwx -R .
!mv php-proxy-app/* .

In [ ]:
!rm -rf ngrok.log
!ngrok tcp 8001 -log=stdout > ngrok.log & php -S localhost:8001